In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
! pip install sudachipy
! pip install sudachidict_core
! pip install neologdn

     |████████████████████████████████| 70 kB 3.2 MB/s 
     |████████████████████████████████| 473 kB 18.5 MB/s 
  Created wheel for sudachipy: filename=SudachiPy-0.5.2-cp37-cp37m-linux_x86_64.whl size=870254 sha256=1fd9f75e48d0dce1e5e0976c29ef37825e02a43612702a7f7c1c3b32a56c11e7
  Stored in directory: /root/.cache/pip/wheels/43/72/0f/1c62895bde30566c65602f15ddbfa0b2bbc273f8c43c190a45
Successfully built sudachipy
  Attempting uninstall: sortedcontainers
    Found existing installation: sortedcontainers 2.4.0
    Uninstalling sortedcontainers-2.4.0:
      Successfully uninstalled sortedcontainers-2.4.0
  Created wheel for sudachidict-core: filename=SudachiDict_core-20210608-py3-none-any.whl size=71421461 sha256=b6026bdd2ba257fbb1dcccad8629ac53bdf72ac83d9ed0fcf6f62bbab20f86e3
  Stored in directory: /root/.cache/pip/wheels/08/3e/74/0beaa92bc46b8d5c14b53d2640f8c3d74cbb33b4ec1fc6c213
Successfully built sudachidict-core
     |████████████████████████████████| 57 kB 1.7 MB/s 
  Created wheel

### Loading Libraries

In [24]:
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
from sudachipy import tokenizer
from sudachipy import dictionary
import nltk
import neologdn
import numpy as np
#nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('words')
import spacy
import re
import string
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
eng_words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /Users/kapeleshh/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [25]:
en_alphabets = ['o','q','w','e','r','t','y','u','i','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m']
sw_spacy.update(en_alphabets)

### Load Data

In [28]:
filepath = "/Users/kapeleshh/PycharmProjects/STS/"
df=  pd.read_csv(filepath + "data/preproc_train.csv")
#df = pd.read_csv(filepath + "data/Ctoken_train.csv")

### Load Data for Japanese stopwords

In [29]:
file1 = open(filepath + "stopwords/Japanese_stopword_list.txt")

# Use this to read file content as a stream:
line = file1.read()
stopJa = line.split()

### Sudachi Tokenization

In [30]:
def surface_sudachitokens(df):
    tokenizer_obj = dictionary.Dictionary().create()
    mode = tokenizer.Tokenizer.SplitMode.C #(Mode = B or C)
    for i in range(df.shape[0]):
        a = df.iloc[i].to_string()
        a = neologdn.normalize(a)
        df['text'][i] = [m.surface() for m in tokenizer_obj.tokenize(a, mode)]
        print(i)
    return df

def normalized_sudachitokens(df):
    tokenizer_obj = dictionary.Dictionary().create()
    mode = tokenizer.Tokenizer.SplitMode.C #(Mode = B or C)
    for i in range(df.shape[0]):
        a = df.iloc[i].to_string()
        a = neologdn.normalize(a)
        df['text'][i] = [m.normalized() for m in tokenizer_obj.tokenize(a, mode)]
        print(i)
    return df

def stopwords_removal(df, en_stopwords, ja_stopwords, en_dictionary):
    for i in range(df.shape[0]):
        #df['text'][i]= [w for w in df['text'][i] if(re.findall(r"([a-zA-Z]+)",w) or re.findall(r"([ぁ-んァ-ン]+)",w) or re.findall(r"([一-龯]+)",w) or re.findall(r"(\d+)",w)) or w.lower() in eng_words]
        df['text'][i]= [w for w in df['text'][i] if(re.findall(r"([ぁ-んァ-ン]+)",w) or re.findall(r"([一-龯]+)",w) or re.findall(r"(\d+)",w)) or w.lower() in en_dictionary or not w.isalpha()]
        df['text'][i]= [w for w in df['text'][i] if(re.findall(r"([a-zA-Z]+)",w) or re.findall(r"([ぁ-んァ-ン]+)",w) or re.findall(r"([一-龯]+)",w) or re.findall(r"(\d+)",w))]
        df['text'][i] = [word for word in df['text'][i] if word not in en_stopwords and word not in string.punctuation and word not in ja_stopwords]
 #      df['text'][i] = [word for word in df['text'][i] if word.lower() in eng_words or not word.isalpha()]
        df['text'][i] = ' '.join(df['text'][i]).split()
        print(i)
    df.text = df.text.apply(lambda y: np.nan if len(y)==0 else y)
    return df

In [ ]:
surface_sudachitokens(dfs)

In [33]:
stopwords_removal(dfs, sw_spacy, stopJa, eng_words)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


/Users/kapeleshh/.conda/envs/STS/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/kapeleshh/.conda/envs/STS/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/kapeleshh/.conda/envs/STS/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

,text
0,NaN
1,"[りょう, むりょう]"
2,NaN
3,"[つぼみ, つい, ちいさく, くさっ, もう, かい, ませ]"
4,NaN
...,...
95,"[そんな, よく, そんな, よく, そんな, よく, そんな, よく, そんな, よく, ..."
96,"[いい, いい, いい, いい, いい, いい, いい, いい, いい, いい, いい, い..."
97,"[うまうま, うまうま, うまうま, うまうま, うまうま, うまうま, うまうま, うまう..."
98,[グッド]
